In [15]:
import gzip
import re
import ast
import pandas as pd

# Extracción de Datos

## Descomprimir y Reconvertir Datos

### Archivo steam_games.json.gz

In [16]:
# Direccion del archivo comprimido
steam_games_content= '../_src/steam_games.json.gz'
# Cargar directamente el archivo JSON comprimido en un DataFrame
steam_games_dataset = pd.read_json(steam_games_content, compression='gzip', lines=True)

Vemos el tamaño del dataframe original

In [17]:
print(f"El DataFrame tiene {steam_games_dataset.shape[0]} filas y {steam_games_dataset.shape[1]} columnas.")

El DataFrame tiene 120445 filas y 13 columnas.


Eliminamos filas nulas

In [18]:
steam_games_dataset.dropna(inplace=True)

Vemos el tamaño del dataframe después de la eliminacion

In [19]:
print(f"El DataFrame tiene {steam_games_dataset.shape[0]} filas y {steam_games_dataset.shape[1]} columnas.")

El DataFrame tiene 22530 filas y 13 columnas.


Constatamos tipos de datos y forma de la tabla

In [20]:
print(steam_games_dataset.dtypes)

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access    float64
id              float64
developer        object
dtype: object


Probar si puedo eliminar duplicados

In [21]:
try:
    #Elimina duplicados
    steam_games_dataset.drop_duplicates(inplace=True)
except TypeError as error: 
    # Si el tipo de la columna no es el que puede eliminar duplicados, 
    # lo reconoce, lo modifica a tuplas, elimina duplicados y devuelve el tipo inicial
    palabra_entre_comillas = re.search(r"'(.*?)'", str(error))
    # Convertir la columna con listas en tuplas
    steam_games_dataset = steam_games_dataset.applymap(lambda x: tuple(x) if isinstance(x, eval(palabra_entre_comillas.group(1))) else x)
    #Elimina duplicados
    steam_games_dataset.drop_duplicates(inplace=True)
    # Revertir el dataframe de tuplas a listas después de eliminar duplicados
    steam_games_dataset = steam_games_dataset.applymap(lambda x: eval(palabra_entre_comillas.group(1))(x) if isinstance(x, tuple) else x)
except Exception as e:
    print(f"Ocurrió un error: {e}. Continúa con los trabajos sobre el dataframe.")

/var/folders/0p/tsl8rn8x1vs7k18zw9s6dh0r0000gn/T/ipykernel_12286/2613242174.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  steam_games_dataset = steam_games_dataset.applymap(lambda x: tuple(x) if isinstance(x, eval(palabra_entre_comillas.group(1))) else x)


/var/folders/0p/tsl8rn8x1vs7k18zw9s6dh0r0000gn/T/ipykernel_12286/2613242174.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  steam_games_dataset = steam_games_dataset.applymap(lambda x: eval(palabra_entre_comillas.group(1))(x) if isinstance(x, tuple) else x)


Vemos el tamaño del dataframe después eliminar duplicados

In [22]:
print(f"El DataFrame tiene {steam_games_dataset.shape[0]} filas y {steam_games_dataset.shape[1]} columnas.")

El DataFrame tiene 22530 filas y 13 columnas.


Guardado del DataFrame a un archivo convertido a .parquet para comodidad en el trabajo de ETL y EDA posterior

In [23]:
steam_games_dataset['price'] = steam_games_dataset['price'].astype(str) # Se hace porque da error de tipo al guardar

In [24]:
# Guardar el df en un archivo parquet
steam_games_dataset.to_parquet('../_data/01_steam_games.parquet', index=False)

### Archivo users_items.json.gz

In [25]:
# Direccion del archivo
users_items_content= '../_src/users_items.json.gz'

# Lista para guardar cada fila
lista_users_items = []

# Descomprimir el archivo y cargar JSON línea por línea
with gzip.open(users_items_content, 'rb') as users_items_gzip:
    for linea in users_items_gzip:
        try:
            # Decodifica la línea y agrega el objeto a la lista
            fila = ast.literal_eval(linea.decode('utf-8'))
            lista_users_items.append(fila)
        except ValueError as e:
            print(f"Error al cargar la línea: {linea}. Error: {e}")

# Convertir la lista a DataFrame
users_items_dataset = pd.DataFrame(lista_users_items)

Vemos el tamaño del dataframe original

In [26]:
print(f"El DataFrame tiene {users_items_dataset.shape[0]} filas y {users_items_dataset.shape[1]} columnas.")

El DataFrame tiene 88310 filas y 5 columnas.


Eliminamos filas nulas

In [27]:
users_items_dataset.dropna(inplace=True)

Vemos el tamaño del dataframe después de la eliminacion

In [28]:
print(f"El DataFrame tiene {users_items_dataset.shape[0]} filas y {users_items_dataset.shape[1]} columnas.")

El DataFrame tiene 88310 filas y 5 columnas.


Constatamos tipos de datos y forma de la tabla

In [29]:
print(users_items_dataset.dtypes)

user_id        object
items_count     int64
steam_id       object
user_url       object
items          object
dtype: object


Probar si puedo eliminar duplicados

In [30]:
try:
    users_items_dataset.drop_duplicates(inplace=True)
except TypeError as error: 
    palabra_entre_comillas = re.search(r"'(.*?)'", str(error))
    print(f"No se pudo eliminar duplicados porque el dataframe es de tipo {palabra_entre_comillas.group(1)}, intentar mas adelante.")
except Exception as e:
    print(f"Ocurrió un error: {e}. Continúa con los trabajos sobre el dataframe.")

No se pudo eliminar duplicados porque el dataframe es de tipo list, intentar mas adelante.


Vemos el tamaño del dataframe después de la eliminacion

In [31]:
print(f"El DataFrame tiene {users_items_dataset.shape[0]} filas y {users_items_dataset.shape[1]} columnas.")

El DataFrame tiene 88310 filas y 5 columnas.


Guardado del DataFrame a un archivo convertido a .parquet para ahorro de espacio y comodidad en el trabajo de ETL y EDA posterior

In [32]:
# Guardar DataFrame en un archivo Parquet
users_items_dataset.to_parquet('../_data/01_users_items.parquet', index=False)

### Archivo user_reviews.json.gz

In [33]:
# Direccion del archivo
user_reviews_content= '../_src/user_reviews.json.gz'

# Lista para guardar cada fila
lista_user_reviews = []

# Descomprimir el archivo y cargar JSON línea por línea
with gzip.open(user_reviews_content, 'rb') as user_reviews_gzip:
    for linea in user_reviews_gzip:
        try:
            # Decodifica la línea y agrega el objeto a la lista
            fila = ast.literal_eval(linea.decode('utf-8'))
            lista_user_reviews.append(fila)
        except ValueError as e:
            print(f"Error al cargar la línea: {linea}. Error: {e}")

# Convertir la lista a DataFrame
user_reviews_dataset = pd.DataFrame(lista_user_reviews)

Vemos el tamaño del dataframe original

In [34]:
print(f"El DataFrame tiene {user_reviews_dataset.shape[0]} filas y {user_reviews_dataset.shape[1]} columnas.")

El DataFrame tiene 25799 filas y 3 columnas.


Eliminamos filas nulas

In [35]:
user_reviews_dataset.dropna(inplace=True)

Vemos el tamaño del dataframe después de la eliminacion

In [36]:
print(f"El DataFrame tiene {user_reviews_dataset.shape[0]} filas y {user_reviews_dataset.shape[1]} columnas.")

El DataFrame tiene 25799 filas y 3 columnas.


Constatamos tipos de datos y forma de la tabla

In [37]:
print(user_reviews_dataset.dtypes)

user_id     object
user_url    object
reviews     object
dtype: object


Probar si puedo eliminar duplicados

In [38]:
try:
    user_reviews_dataset.drop_duplicates(inplace=True)
except TypeError as error: 
    palabra_entre_comillas = re.search(r"'(.*?)'", str(error))
    print(f"No se pudo eliminar duplicados porque el dataframe es de tipo {palabra_entre_comillas.group(1)}, intentar mas adelante.")
except Exception as e:
    print(f"Ocurrió un error: {e}. Continúa con los trabajos sobre el dataframe.")

No se pudo eliminar duplicados porque el dataframe es de tipo list, intentar mas adelante.


Vemos el tamaño del dataframe después de la eliminacion

In [39]:
print(f"El DataFrame tiene {user_reviews_dataset.shape[0]} filas y {user_reviews_dataset.shape[1]} columnas.")

El DataFrame tiene 25799 filas y 3 columnas.


Guardado del DataFrame a un archivo convertido a .parquet para ahorro de espacio y comodidad en el trabajo de ETL y EDA posterior

In [40]:
# Guardar DataFrame en un archivo Parquet
user_reviews_dataset.to_parquet('../_data/01_user_reviews.parquet', index=False)

Se eligen guardar todos los dataframes en formato paquet ya que este es muy liviano y eficiente para manejar datos. Además ayudara en subsiguientes trabajos y finalmente subirlos a FastAPI y render.